## APPLICATION TEMPLATE OVERVIEW - GLOBAL DAILY MEAN TEMPERATURE

Templates are built upon `c3s_eqc_automatic_quality_control` (https://github.com/bopen/c3s-eqc-automatic-quality-control)

`$HOME/.cdsapirc` is used by default.\
If you need to use different credentials, uncomment the lines in the following cell and edit the path to `.cdsapirc`.\
See: https://cds.climate.copernicus.eu/api-how-to

In [ ]:
# import os
# os.environ["CDSAPI_RC"] = os.path.expanduser("~/lastname_firstname/.cdsapirc")

Import libraries

In [ ]:
import warnings

import plotly.express as px
import xarray as xr
from c3s_eqc_automatic_quality_control import diagnostics, download

warnings.filterwarnings("ignore")

### PROCESSING with Xarray: GLOBAL DAILY MEAN TEMPERATURE 2021-2022

APPLICATION SUMMARY:

- Data requests definition

- Processing:

  - chunked download
  - apply transformation to each chunk
  - cache the result on each chunk
  - merge all the chunks

- Plot the result

#### REQUEST DEFINITION

For request definition we use **`download.update_request_date`**.

It allows to generate the requests for a given period `[start, stop]` if stop is `None` then compute the stop month as follows:
if `current day > switch_month_day`: then `stop_month = current_month - 1`
else `stop_month = current_month - 2`

Returns the request or the list of requests for the input period.

In [ ]:
collection_id = "reanalysis-era5-single-levels"

request = {
    "product_type": "reanalysis",
    "format": "grib",
    "time": ["06", "18"],
    "variable": "2m_temperature",
}
start = "2022-11"
stop = None

In [ ]:
requests = download.update_request_date(
    request, start=start, stop=stop, switch_month_day=9
)
requests

#### DOWNLOAD and PROCESSING

The processing will be perform as follows:

**Definition of the transformation function `transform_func`** to be applied to each single chunk

**Chunk definition**: define how to chunk the request along time dimension (year, month, day):
e.g. `chunks={"year": 1,"month": 1}`. Alternatively, use `split_all=True` to chunk as much as possible.

**Download and processing**: for each chunk in time

```
- Download chunk
- Cache the downloaded chunk (optional)
- Apply the operation `transform_func` to the chunk
- Cache transformed chunk, `transform_func(chunks)` (optional)
```

Merge all the transformed chunk

#### Define `f`

In [ ]:
def spatial_daily_mean(ds: xr.Dataset) -> xr.Dataset:
    ds = diagnostics.spatial_weighted_mean(ds)
    return ds.resample(forecast_reference_time="1D").mean("forecast_reference_time")

#### Define `chunks`

In [ ]:
chunks = {"year": 1, "month": 1}

#### Perform the download and processing

In [ ]:
daily_global_t2m = download.download_and_transform(
    collection_id,
    requests,
    chunks=chunks,
    transform_func=spatial_daily_mean,
)

daily_global_t2m

#### Plot result

In [ ]:
daily_global_t2m = daily_global_t2m.squeeze()
fig = px.line(
    x=daily_global_t2m["forecast_reference_time"],
    y=daily_global_t2m["t2m"] - 273.15,
)
fig.update_layout(
    xaxis_title="forecast_reference_time",
    yaxis_title="t2m",
    title="GLOBAL DAILY MEAN TEMPERATURE 2021-2022",
)
fig.show()

#### Save figure

In [ ]:
fig.write_image("ERA5_global_daily_mean_temperature_2021-2022.png")